In [ ]:
import os
import math
import torch
import torchvision
import collections
import numpy as np
import pandas as pd
import cv2
from math import exp
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from skimage.io import imread, imsave
from skimage import transform
from skimage.transform import rotate, AffineTransform
from skimage.io import imread
from skimage.color import rgb2gray, gray2rgb
from scipy import ndimage
import skimage.io as io
from PIL import Image

In [ ]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        


In [ ]:
createFolder('/content/result/')
createFolder('/content/result/predicted/')
createFolder('/content/result/truth/')


In [ ]:
# Dividing input image into 128 patch size
size = 128
stride = 128

k=1
path1='/content/result/predicted/'
path2='/content/result/truth/'
for i,data in enumerate(trainset):
  inputs,labels = data
  inputs=inputs.contiguous().view(inputs.size(0), inputs.size(1),inputs.size(2))
  labels=labels.contiguous().view(labels.size(0), labels.size(1),labels.size(2))
  inputs = inputs.unfold(1, size, stride).unfold(2, size, stride)
  labels = labels.unfold(1, size, stride).unfold(2, size, stride)
  inputs = inputs.contiguous().view(inputs.size(0), -1, size, size)
  labels = labels.contiguous().view(labels.size(0), -1,size, size)
  inputs =inputs.permute(1,0,2,3)
  labels =labels.permute(1,0,2,3)
  for i in range(inputs.shape[0]):
    p=inputs[i,:]
    im = transforms.ToPILImage()(p)
    im.save(path1+"{}".format(k)+".png")
    p=labels[i,:]
    im = transforms.ToPILImage()(p)
    im.save(path2+"{}".format(k)+".png")
    k+=1
  


In [ ]:
dice_result=[]
for i in range(1,k):
    inputs = Image.open(path1+str(i)+'.png')
    target = Image.open(path2+str(i)+'.png')
    t=dice(inputs,target)
    dice_result.append(t)


In [ ]:
porosity=[]
for i in range(1,k):
    inputs = Image.open(path1+str(i)+'.png')
    target = Image.open(path2+str(i)+'.png')
    inputs = np.asarray(inputs).astype(np.bool)
    target= np.asarray(target).astype(np.bool)
    p=inputs.sum()/(128*128)
    q=target.sum()/(128*128)
    porosity.append([p,q])


In [ ]:
porosity=pd.DataFrame(porosity)
porosity.to_csv('porostity.csv')

dice_result=pd.DataFrame(dice_result)
dice_result.to_csv('dice_result.csv')

In [ ]:
import shutil
shutil.make_archive('result', 'zip', '/content/result')